# Data format transformation from txt to HDF5

1. Variable "dataset_name" needs to be modified to match the folder name of dataset 
2. Variable "dataset_name" needs to be modified to match the folder name of dataset 

In [1]:
import os
import numpy as np
import h5py
import glob
import pandas as pd
import torch


def read_files_array(filename):
    row_data = pd.read_csv(filename,sep=' ', header=None)
    row_data = row_data.iloc[0:,2].values    
    matrix = row_data.astype('float').reshape(21,21)
    return matrix


dataset_name = 'data_bc' # Needs to be modified to match the folder name of dataset 
fname_h5 = dataset_name + '_shuffle.hdf5'
fname_h5_all = dataset_name + '_all.hdf5'
path = './'+ dataset_name +'/'
fname_lib = path + 'dataset_DKtGeo.txt'
fld_in = path + 'input/'
fld_out= path + 'output/'

# Dataset shuffle settings
ratio_test = 0.25 
random_seed = 42
shuffle_dataset = False

data_lib = pd.read_csv(fname_lib, sep="\t", header=None).values
dataset_size = len(data_lib)
indices = list(range(dataset_size))
split = int(np.floor(ratio_test * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    train_indices, test_indices = indices[split:], indices[:split]
    
    train_input_shape = (len(train_indices), 4, 21, 21)
    test_input_shape = (len(test_indices), 4, 21, 21)
    train_output_shape = (len(train_indices), 1, 21, 21)
    test_output_shape = (len(test_indices), 1, 21, 21)

    # open a hdf5 file and create earrays

    # create a new HDF5 file
    hdf5_file = h5py.File(fname_h5, mode='w')

    # # create groups in HDF5 file
    # f.create_group('/train_input') 
    # f.create_group('/train_output')
    # f.create_group('/test_input') 
    # f.create_group('/test_output')


    hdf5_file.create_dataset("train_input", train_input_shape, np.float64)
    # hdf5_file.create_dataset("val_input", val_input_shape, np.float64)
    hdf5_file.create_dataset("test_input", test_input_shape, np.float64)

    hdf5_file.create_dataset("train_output", train_output_shape, np.float64)
    # hdf5_file.create_dataset("val_output", val_output_shape, np.float64)
    hdf5_file.create_dataset("test_output", test_output_shape, np.float64)

    count = 0
    for index in train_indices:
        file_num = data_lib[index,0]
        t = data_lib[index,3]
        k = data_lib[index,2]
        d = data_lib[index,1]
        geo = data_lib[index,8]
        paraset = data_lib[index,9]

        filename_output = fld_out + "mesh_"+str(int(file_num))+".txt"
            
        # matrix_input = torch.zeros([4,21,21], dtype = torch.float)
        matrix_input = np.zeros((4, 21, 21), dtype=np.float64)
        filename_input = fld_in + "geometry_"+str(int(geo))+ "_" +str(int(paraset)) + "_input.txt"
        matrix_input[0] = read_files_array(filename_input )
        matrix_input[1] = t
        matrix_input[2] = k
        matrix_input[3] = d

        # matrix_output = torch.zeros([1,21,21], dtype = torch.float)
        matrix_output = np.zeros((1, 21, 21), dtype=np.float64)
        matrix_output = read_files_array(filename_output)

        hdf5_file["train_input"][count, ...] = matrix_input
        hdf5_file["train_output"][count, ...] = matrix_output
        count += 1

    count = 0
    for index in test_indices:
        file_num = data_lib[index,0]
        t = data_lib[index,3]
        k = data_lib[index,2]
        d = data_lib[index,1]
        geo = data_lib[index,8]
        paraset = data_lib[index,9]

        filename_output = fld_out + "mesh_"+str(int(file_num))+".txt"
            
        # matrix_input = torch.zeros([4,21,21], dtype = torch.float)
        matrix_input = np.zeros((4, 21, 21), dtype=np.float64)
        filename_input = fld_in + "geometry_"+str(int(geo))+ "_" +str(int(paraset)) + "_input.txt"
        matrix_input[0] = read_files_array(filename_input )
        matrix_input[1] = t
        matrix_input[2] = k
        matrix_input[3] = d

        # matrix_output = torch.zeros([1,21,21], dtype = torch.float)
        matrix_output = np.zeros((1, 21, 21), dtype=np.float64)
        matrix_output = read_files_array(filename_output)

        hdf5_file["test_input"][count, ...] = matrix_input
        hdf5_file["test_output"][count, ...] = matrix_output    
        
        count += 1

    hdf5_file.close()
else:
    hdf5_file = h5py.File(fname_h5_all, mode='w')
    
    input_shape = (dataset_size, 4, 21, 21)
    output_shape = (dataset_size, 1, 21, 21)

    hdf5_file.create_dataset("input", input_shape, np.float64)
    hdf5_file.create_dataset("output", output_shape, np.float64)


    count = 0
    for index in range(0,dataset_size):
        file_num = data_lib[index,0]
        t = data_lib[index,3]
        k = data_lib[index,2]
        d = data_lib[index,1]
        geo = data_lib[index,8]
        paraset = data_lib[index,9]

        filename_output = fld_out + "mesh_"+str(int(file_num))+".txt"
            
        # matrix_input = torch.zeros([4,21,21], dtype = torch.float)
        matrix_input = np.zeros((4, 21, 21), dtype=np.float64)
        filename_input = fld_in + "geometry_"+str(int(geo))+ "_" +str(int(paraset)) + "_input.txt"
        matrix_input[0] = read_files_array(filename_input )
        matrix_input[1] = t
        matrix_input[2] = k
        matrix_input[3] = d

        # matrix_output = torch.zeros([1,21,21], dtype = torch.float)
        matrix_output = np.zeros((1, 21, 21), dtype=np.float64)
        matrix_output = read_files_array(filename_output)

        hdf5_file["input"][count, ...] = matrix_input
        hdf5_file["output"][count, ...] = matrix_output
        count += 1

    hdf5_file.close()


# Data format transformation from txt to HDF5 (For old data without different BCs)

In [100]:
import os
import numpy as np
import h5py
import glob
import pandas as pd
import torch


def read_files_array(filename):
    row_data = pd.read_csv(filename,sep=' ', header=None)
    row_data = row_data.iloc[0:,2].values    
    matrix = row_data.astype('float').reshape(21,21)
    return matrix


dataset_name = 'data'
fname_h5 = dataset_name + '_shuffle.hdf5'
fname_h5_all = dataset_name + '_all.hdf5'
path = './'+ dataset_name +'/'
fname_lib = path + 'dataset_DKtGeo.txt'
fld_in = path + 'input/'
fld_out= path + 'output/'

# Shuffle dataset
ratio_test = 0.25
random_seed = 42
shuffle_dataset = False

data_lib = pd.read_csv(fname_lib, sep="\t", header=None).values
dataset_size = len(data_lib)
print(dataset_size)
indices = list(range(dataset_size))
split = int(np.floor(ratio_test * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    train_indices, test_indices = indices[split:], indices[:split]
    
    train_input_shape = (len(train_indices), 4, 21, 21)
    test_input_shape = (len(test_indices), 4, 21, 21)
    train_output_shape = (len(train_indices), 1, 21, 21)
    test_output_shape = (len(test_indices), 1, 21, 21)

    # open a hdf5 file and create earrays

    # create a new HDF5 file
    hdf5_file = h5py.File(fname_h5, mode='w')


    hdf5_file.create_dataset("train_input", train_input_shape, np.float64)
    # hdf5_file.create_dataset("val_input", val_input_shape, np.float64)
    hdf5_file.create_dataset("test_input", test_input_shape, np.float64)

    hdf5_file.create_dataset("train_output", train_output_shape, np.float64)
    # hdf5_file.create_dataset("val_output", val_output_shape, np.float64)
    hdf5_file.create_dataset("test_output", test_output_shape, np.float64)

    count = 0
    for index in train_indices:
        file_num = data_lib[index,0]
        t = data_lib[index,3]
        k = data_lib[index,2]
        d = data_lib[index,1]
        geo = data_lib[index,8]

        filename_output = fld_out + "mesh_"+str(int(file_num))+".txt"
            
        # matrix_input = torch.zeros([4,21,21], dtype = torch.float)
        matrix_input = np.zeros((4, 21, 21), dtype=np.float64)
        filename_input = fld_in + "geometry_"+str(int(geo))+ "_input.txt"
        matrix_input[0] = read_files_array(filename_input )
        matrix_input[1] = t
        matrix_input[2] = k
        matrix_input[3] = d

        # matrix_output = torch.zeros([1,21,21], dtype = torch.float)
        matrix_output = np.zeros((1, 21, 21), dtype=np.float64)
        matrix_output = read_files_array(filename_output)

        hdf5_file["train_input"][count, ...] = matrix_input
        hdf5_file["train_output"][count, ...] = matrix_output
        count += 1

    count = 0
    for index in test_indices:
        file_num = data_lib[index,0]
        t = data_lib[index,3]
        k = data_lib[index,2]
        d = data_lib[index,1]
        geo = data_lib[index,8]

        filename_output = fld_out + "mesh_"+str(int(file_num))+".txt"
            
        # matrix_input = torch.zeros([4,21,21], dtype = torch.float)
        matrix_input = np.zeros((4, 21, 21), dtype=np.float64)
        filename_input = fld_in + "geometry_"+str(int(geo))+ "_input.txt"
        matrix_input[0] = read_files_array(filename_input )
        matrix_input[1] = t
        matrix_input[2] = k
        matrix_input[3] = d

        # matrix_output = torch.zeros([1,21,21], dtype = torch.float)
        matrix_output = np.zeros((1, 21, 21), dtype=np.float64)
        matrix_output = read_files_array(filename_output)

        hdf5_file["test_input"][count, ...] = matrix_input
        hdf5_file["test_output"][count, ...] = matrix_output    
        
        count += 1

    hdf5_file.close()
else:
    hdf5_file = h5py.File(fname_h5_all, mode='w')

    input_shape = (dataset_size, 4, 21, 21)
    output_shape = (dataset_size, 1, 21, 21)
    
    hdf5_file.create_dataset("input", input_shape, np.float64)
    hdf5_file.create_dataset("output", output_shape, np.float64)


    count = 0
    for index in range(0,dataset_size):
        file_num = data_lib[index,0]
        t = data_lib[index,3]
        k = data_lib[index,2]
        d = data_lib[index,1]
        geo = data_lib[index,8]

        filename_output = fld_out + "mesh_"+str(int(file_num))+".txt"
            
        # matrix_input = torch.zeros([4,21,21], dtype = torch.float)
        matrix_input = np.zeros((4, 21, 21), dtype=np.float64)
        filename_input = fld_in + "geometry_"+str(int(geo)) + "_input.txt"
        matrix_input[0] = read_files_array(filename_input )
        matrix_input[1] = t
        matrix_input[2] = k
        matrix_input[3] = d

        # matrix_output = torch.zeros([1,21,21], dtype = torch.float)
        matrix_output = np.zeros((1, 21, 21), dtype=np.float64)
        matrix_output = read_files_array(filename_output)

        hdf5_file["input"][count, ...] = matrix_input
        hdf5_file["output"][count, ...] = matrix_output
        count += 1

    hdf5_file.close()


240190
